#### 1. Import pandas library

In [1]:
import pandas as pd

#### 2. Import users table:

In [20]:
users = pd.read_csv('/Users/joananogueira/Desktop/Ironhack /Labs/Week2/lab-data-cleaning/your-code/users_table.csv')

#### 3. Rename Id column to userId

In [21]:
users.rename(columns={'Id':'userId'}, inplace = True)

#### 4. Import posts table:

In [22]:
posts = pd.read_csv('/Users/joananogueira/Desktop/Ironhack /Labs/Week2/lab-data-cleaning/your-code/posts_table.csv')

#### 5. Rename Id column to postId and OwnerUserId to userId

In [23]:
posts.rename(columns={'Id':'postId', 'OwnerUserId':'userId'}, inplace = True)

#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [24]:
users_new = users[['userId', 'Reputation','Views', 'UpVotes', 'DownVotes']]
posts_new = posts[['postId', 'Score', 'userId', 'ViewCount', 'CommentCount']]

#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [37]:
merged_df = users_new.merge(posts_new)

#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [38]:
merged_df.isnull().sum()

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [39]:
merged_df['ViewCount'] = merged_df['ViewCount'].fillna(0)

In [40]:
merged_df.isnull().sum()

userId          0
Reputation      0
Views           0
UpVotes         0
DownVotes       0
postId          0
Score           0
ViewCount       0
CommentCount    0
dtype: int64

#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [45]:
merged_df['ViewCount'] = merged_df['ViewCount'].astype('int64')
merged_df


,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,0,0
1,-1,1,0,5007,1920,8576,0,0,0
2,-1,1,0,5007,1920,8578,0,0,0
3,-1,1,0,5007,1920,8981,0,0,0
4,-1,1,0,5007,1920,8982,0,0,0
...,...,...,...,...,...,...,...,...,...
38957,45934,11,1,0,0,34003,1,115,2
38958,46192,36,1,0,0,40667,5,326,2
38959,46522,235,13,27,1,17461,3,166,0
38960,52371,221,2,0,0,27237,24,3357,5
